### Setting up directory

In [48]:
import torch
import pandas as pd
from transformers import pipeline
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report


print(torch.cuda.is_available())  
device = 0 if torch.cuda.is_available() else -1
print(f"device={device}")

True
device=0


### loading the model

In [49]:
model_name="cardiffnlp/twitter-roberta-base-sentiment" #approx 500mb size
sentiment_pipeline = pipeline("sentiment-analysis", model=model_name, device=device)

Device set to use cuda:0


### predicting on data

### checking model accuracy 

In [50]:
df = pd.read_parquet("data/train.parquet", engine="pyarrow")
# df=df[:100].copy()
results=sentiment_pipeline(df["text"].to_list())

df['sentiment_label_code'] = [result['label'] for result in results]
df['sentiment_score'] = [result['score'] for result in results]
label_mapping = {"LABEL_0": "negative", "LABEL_1": "neutral", "LABEL_2": "positive"}
df['sentiment_label'] = df['sentiment_label_code'].map(label_mapping)

In [51]:
y_pred = df['sentiment_label']
y_true = df['label_text']

In [52]:
accuracy = accuracy_score(y_true, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, labels=["positive", "neutral", "negative"], average='weighted', zero_division=0)

print("Accuracy:", accuracy)
print("\nClassification Report:")
print(classification_report(y_true, y_pred, labels=["positive", "neutral", "negative"]))

Accuracy: 0.7209737223923908

Classification Report:
              precision    recall  f1-score   support

    positive       0.70      0.87      0.78      9685
     neutral       0.74      0.57      0.64     12548
    negative       0.72      0.78      0.75      8782

    accuracy                           0.72     31015
   macro avg       0.72      0.74      0.72     31015
weighted avg       0.72      0.72      0.71     31015

